In [ ]:
import os
import math
import random
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import matplotlib.animation as animation

from src.perceptron import Perceptron
from src.adaline import Adaline

%load_ext autoreload
%autoreload 2

In [ ]:
dataset_path = './resources/winequality-red.csv'

try:
    wine_data = pd.read_csv(dataset_path, sep=';')
except FileNotFoundError:
    print('[-] Set `dataset_path` with correct value')

----------------------------------
# V.1 Exploring the green reds

### a) Write a function that will plot a scatterplot matrix of your red wine data

In [ ]:
def plot_scatter_matrix(wine_data, good_threshold, bad_threshold, save_plot=False):
    """
    Plots a scatterplot matrix of data. Samples with quality over `good_threshold`
    are plotted as one color and samples below `bad_threshold` as another.

    :param pd.DataFrame wine_data:
    :param int good_threshold:
    :param int bad_threshold:

    :returns: result matplotlib.pyplot.figure object
    """
    num_samples, num_features = wine_data.shape

    fig, axes = plt.subplots(nrows=num_features,
                             ncols=num_features, figsize=(18,18))
    fig.subplots_adjust(hspace=0, wspace=0)
    
    for ax in axes.flat:
        ax.xaxis.set_visible(False)
        ax.yaxis.set_visible(False)
    
    feature_names = wine_data.columns
    for i, label in enumerate(feature_names):
        axes[i, i].annotate(label, (0.5, 0.5), xycoords='axes fraction',
                            ha='center', va='center')
    
    good_wines = wine_data[(wine_data['quality'] > good_threshold)]
    bad_wines = wine_data[(wine_data['quality'] < bad_threshold)]
    
    for i in range(num_features):
        for j in range(i + 1, num_features):
            axes[i, j].scatter(good_wines.iloc[:, j], good_wines.iloc[:, i], c=['g'], marker='.')
            axes[i, j].scatter(bad_wines.iloc[:, j], bad_wines.iloc[:, i], c=['r'], marker='.')
            axes[j, i].scatter(good_wines.iloc[:, i], good_wines.iloc[:, j], c=['g'], marker='.')
            axes[j, i].scatter(bad_wines.iloc[:, i], bad_wines.iloc[:, j], c=['r'], marker='.')

    if save_plot:
        plt.savefig('./scatterplot-matrix.png')
    
    return fig

In [ ]:
fig = plot_scatter_matrix(wine_data, 7, 4, True)
plt.show(fig)

### b) Which factors do you think will be most useful to your perceptron for distinguishing high quality vs. low quality wines? Why?

Perceptron, being a linear model, works best on linearly separable data.

- **'alcohol'**, **'pH'**, **'sulphates'** look like good features for classification; 
- **'density'**, **'chlorides'** are bad features and most likely won't help;

----------------------------------------------------------
# V.2 Learning to perceptron

### a) & b) Implementing Perceptron
- Implement Rosenblatt perceptron with randomly initialized weights and bias, heaviside step activation function
- Train your perceptron on 2 chemical factors (for example, alcohol and pH) and only use wines with a score of 8 or higher and wines with a score of 3 or lower

In [ ]:
# Take the wine as good, if it's quality > 5
wine_data = wine_data.assign(goodness=pd.Series(wine_data['quality'] > 5))

# selecting the subset from `wine_data`
# only use wines with a score of 8 or higher and wines with a score of 3 or lower
features = ['volatile acidity', 'alcohol', 'quality', 'goodness']
selected_wine_data = wine_data[(wine_data['quality'] > 7) | (wine_data['quality'] < 4)][features]
selected_wine_data = selected_wine_data.reset_index(drop=True)

X = selected_wine_data.loc[:, ['volatile acidity', 'alcohol']]
Y = selected_wine_data['goodness'].values

print('{} samples selected:\n'.format(selected_wine_data.shape[0]))
print(selected_wine_data)

In [ ]:
def array_init(shape, mode='rand'):
    """
    :param shape: an int or tuple, shape of the array to init
    :param mode: 'rand' | 'zeros' | 'ones'
    """
    
    if mode not in ('rand', 'zeros', 'ones'):
        raise ValueError('invalid mode')
    
    new_shape = shape
    if isinstance(shape, int):
        new_shape = (shape, 1)
    
    if isinstance(shape, tuple) and len(shape) == 1:
        new_shape = (shape[0], 1)
    
    base_dict = {}
    for i in range(new_shape[1]):
        if mode == 'rand':
            base_dict[i] = [0.0001 * random.uniform(-1, 1) for i in range(new_shape[0])]
        elif mode == 'zeros':
            base_dict[i] = new_shape[0] * [0.0]
        elif mode == 'ones':
            base_dict[i] = new_shape[0] * [1.0]
    
    df = pd.DataFrame.from_dict(base_dict)
    
    if isinstance(shape, int) or isinstance(shape, tuple) and len(shape) == 1:
        return df.values.squeeze()
    return df.values

In [ ]:
p = Perceptron(lr=0.005)

train_stats = p.train(X.values, Y, epochs=1000, verbose=True, seed=1699)

### c) Training procedure visualization

In [ ]:
def draw_num_errors(ax, performance, epoch):
    epochs = [elem[0] for elem in performance[:epoch + 1]]
    epoch_errors = [elem[1] for elem in performance[:epoch + 1]]

    ax.plot(epochs, epoch_errors)
    ax.set_xlim([0, len(performance)])
    ax.set_title('Errors as a function of epochs')
    ax.set_xlabel('epoch')
    ax.set_ylabel('classification errors')


def draw_decision_boundary(ax, performance, epoch, wine_data, features):
    x_min = wine_data.loc[:, features[0]].min() - 0.15
    x_max = wine_data.loc[:, features[0]].max() + 0.15
    y_min = wine_data.loc[:, features[1]].min() - 0.15
    y_max = wine_data.loc[:, features[1]].max() + 0.15
    
    ax.set_title('Decision boundary at epoch {}'.format(epoch))
    ax.set_xlabel(features[0])
    ax.set_xlim([x_min, x_max])
    ax.set_ylabel(features[1])
    ax.set_ylim([y_min, y_max])
    
    w2, w1 = performance[epoch][2]
    b = performance[epoch][3]
    slope = -w1/w2
    intercept = -b/w2
    x_coords = range(int(x_min) - 1, int(x_max) + 2)
    y_coords = slope * x_coords + intercept

    ax.plot(x_coords, y_coords, 'b--', label='Decision boundary')
    ax.fill_between(x_coords, y_coords, y_min, color='#99ff99')
    ax.fill_between(x_coords, y_coords, y_max, color='#ff9999')


def draw_scatter(ax, wine_data, features, good_thresh, bad_thresh):
    good_wines = wine_data[(wine_data['quality'] > good_thresh)]
    bad_wines = wine_data[(wine_data['quality'] < bad_thresh)]

    ax.scatter(good_wines.loc[:, features[0]], good_wines.loc[:, features[1]],
                    c=['g'], label='good wines (> {} score)'.format(good_thresh))
    ax.scatter(bad_wines.loc[:, features[0]], bad_wines.loc[:, features[1]],
                    c=['r'], label='bad wines (< {} score)'.format(bad_thresh))
    ax.legend(bbox_to_anchor=(1.05, 1), loc=2)

    
def plot_performance(performance, wine_data, features, good_thresh, bad_thresh,
                     epoch=-1, save_plot=False, save_name='./train_stats.png'):
    """
    Plot the performance of perceptron or adaline.
    This function will produce a two plot figure:
    1) Number of classification errors as a function of epochs
    2) Decision boundary for two factors
    """
    fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(15,5))
    
    if epoch > len(performance) - 1:
        raise ValueError('number of epochs should be less than {}'.format(len(performance)))
    
    if len(features) != 2:
        raise ValueError('number of features should be 2')
    
    if epoch == -1:
        epoch = len(performance) - 1

    draw_num_errors(axes[0], performance, epoch)    
    draw_decision_boundary(axes[1], performance, epoch, wine_data, features)
    draw_scatter(axes[1], wine_data, features, good_thresh, bad_thresh)
    
    if save_plot:
        plt.savefig(save_name)
    
    return fig

In [ ]:
fig = plot_performance(train_stats, selected_wine_data, ['alcohol', 'volatile acidity'], 7, 4, -1, False)
plt.show(fig)

---
# VI.2 Do perceptrons dream of electric sheep?

In [ ]:
def save_animation(performance, data, features, good_thresh, bad_thresh,
                   num_frames=-1, name='performance_animation.html'):
    if len(features) != 2:
        raise ValueError('number of features should be 2')
    
    if num_frames > len(performance) - 1:
            raise ValueError('num_frames should be less than {}'.format(len(performance)))
    
    if num_frames == -1:
        num_frames = len(performance) - 1

    fig = plt.figure(figsize=(15,5), dpi=150)
    ax = plt.gca()
    ax.xaxis.set_visible(False)
    ax.yaxis.set_visible(False)

    def init():
        imobj.set_data(array_init((360, 1000), mode='zeros'))

        return imobj,

    def animate(i, performance, data, features, good_thresh, bad_thresh):
        fname = './train_stats_{}.png'.format(i)
        plot_performance(performance, data, features, good_thresh, bad_thresh,
                                   epoch=i, save_plot=True, save_name=fname)
        img = mpimg.imread(fname)[-1::-1]
        imobj.set_data(img)
        os.remove(fname)

        return imobj,

    imobj = ax.imshow(array_init((360, 1000), mode='zeros'), interpolation='bilinear',
                      origin='lower', alpha=1.0, zorder=1, aspect=1)

    anim = animation.FuncAnimation(fig, animate, init_func=init,
                                   fargs=(performance, data, features, good_thresh, bad_thresh),
                                   repeat=False, frames=range(num_frames),
                                   interval=120, blit=True)
    anim.save(name)
    print('[+] Animation saved as {}'.format(name))

---

### d) Feature Scaling

`x* = (x - average(x)) / (max(x) - min(x))`

In [ ]:
def normalize_series(series):
    """
    Perform mean normalization of pandas.Series
    """

    if not isinstance(series, pd.Series):
        raise ValueError('input argument is not an instance of pandas.Series class')

    return (series - series.mean()) / (series.max() - series.min()) 

In [ ]:
selected_wine_data['volatile acidity'] = normalize_series(selected_wine_data['volatile acidity'])
selected_wine_data['alcohol'] = normalize_series(selected_wine_data['alcohol'])

X = selected_wine_data.loc[:, ['volatile acidity', 'alcohol']]
Y = selected_wine_data['goodness'].values

In [ ]:
p = Perceptron(lr=0.005)

train_stats = p.train(X.values, Y, 100, verbose=True, seed=1699)

In [ ]:
fig = plot_performance(train_stats, selected_wine_data, ['alcohol', 'volatile acidity'], 7, 4, -1, False)
plt.show(fig)

----------------------------------
# V.3 My fair ADALINE

### a) Marvin notices and chides you for torturing your perceptron. Why?

Let's try to distinguish between wines with a score of 4 and lower, and wines with a score of 7 and higher. We'll see, that perceptron does not learn on this data.

It happens because single layer perceptrons are only capable of learning linearly separable patterns. The perceptron learning algorithm does not terminate if the learning set is not linearly separable, it will just never reach a point where all vectors are classified properly.

In [ ]:
features = ['volatile acidity', 'alcohol', 'quality', 'goodness']
selected_wine_data = wine_data[(wine_data['quality'] > 6) | (wine_data['quality'] < 5)][features]
selected_wine_data = selected_wine_data.reset_index(drop=True)

selected_wine_data['volatile acidity'] = normalize_series(selected_wine_data['volatile acidity'])
selected_wine_data['alcohol'] = normalize_series(selected_wine_data['alcohol'])

X = selected_wine_data.loc[:, ['volatile acidity', 'alcohol']]
Y = selected_wine_data['goodness']

print('{} samples selected:\n'.format(X.shape[0]))
print(X[:10])
print('...')

In [ ]:
p = Perceptron(lr=0.001)

train_stats = p.train(X.values, Y, 0, verbose=True)

In [ ]:
fig = plot_performance(train_stats, selected_wine_data, ['alcohol', 'volatile acidity'], 6, 5, -1, False)
plt.show(fig)

### b) & c) Implementing ADALINE (Adaptive Linear Neuron)

Adaline is similar to Perceptron and also belongs to the class of single layer binary classifiers. But there are some differences:
- Adaline cost function is Sum Squared Error (SSE) instead of normal error of each training sample in Perceptron;
- Adaline tries to minimize the cost function using Gradient Descent in contrast to Perceptron learning rule. This is an advantage of Adaline as it allows to calculate the error based on continous values rather than binary value;

### d) Find a good learning rate for your ADALINE

In [ ]:
# trying random search for learning rate tuning

best_lr = 0
min_errors = 999999
for n in range(20):
    lr = round(random.uniform(0.001, 0.05), 5)
    ad = Adaline(lr)
    
    train_stats = ad.train(X.values, Y, 300, 'batch', verbose=False)
    num_errors = min([elem[1] for elem in train_stats])
    last_error = train_stats[-1][1]
    
    print('try: {}, lr: {}, last error: {}, {} min errors,'.format(n, lr, last_error, min_errors))
    
    if num_errors < min_errors:
        min_errors = num_errors
        best_lr = lr

print("Best learnig rate: {}, num errors: {}".format(best_lr, min_errors))

In [ ]:
ad = Adaline(lr=best_lr)

train_stats = ad.train(X.values, Y, 301, 'batch', verbose=True)

In [ ]:
fig = plot_performance(train_stats, selected_wine_data, ['alcohol', 'volatile acidity'], 6, 5, -1, False)
plt.show(fig)

---
## V.4 Advanced wine sampling and resampling

### a) Write a function that uses the holdout method to partition the red wine data into a training and a validation set

In [ ]:
def train_test_split(wine_data, test_size=0.7):
    train_set = wine_data.sample(frac=test_size)
    test_set = wine_data.drop(train_set.index)

    print('Train set size: {}'.format(train_set.shape[0]))
    print('Test set size: {}'.format(test_set.shape[0]))
    
    return (train_set, test_set)

In [ ]:
train_set, test_set = train_test_split(selected_wine_data, test_size=0.7)
X_train = train_set.loc[:, ['volatile acidity', 'alcohol']]
Y_train = train_set['goodness']

X_test = test_set.loc[:, ['volatile acidity', 'alcohol']]
Y_test = test_set['goodness']

In [ ]:
# testing our best model on test_set

ad.evaluate_accuracy(X_test.values, Y_test)

### b) Write a function that generates a k-fold cross-validation dataset from the red wine data

In [ ]:
def k_fold_split(data, k, shuffle=True):
    folds = []
    
    if shuffle:
        data = data.sample(frac=1).reset_index(drop=True)
    
    for i in range(k):
        fold_size = data.shape[0] // k + 1 if i < data.shape[0] % k else data.shape[0] // k
        test_data = data.iloc[i * fold_size: (i + 1) * fold_size, :]
        train_data = data.iloc[data.index.difference(test_data.index), :]
        folds.append((train_data, test_data))
    
    return folds

In [ ]:
num_folds = 9
folds = k_fold_split(selected_wine_data, 9)

print('train, val lengths for {} folds: {}'.format(
    num_folds, [(len(train), len(val)) for train, val in folds]))

In [ ]:
def visualize_folds(folds, features):
    fig, axes = plt.subplots(ncols=len(folds), figsize=(13,5))
    
    for i, fold in enumerate(folds):
        if i != 0:
            axes[i].yaxis.set_visible(False)
        axes[i].scatter(fold[0].loc[:, features[0]], fold[0].loc[:, features[1]],
                    c=['g'], label='training data')
        axes[i].scatter(fold[1].loc[:, features[0]], fold[1].loc[:, features[1]],
                        c=['r'], label='test data')
        
        axes[i].set_xlabel(features[0])
        axes[i].set_ylabel(features[1])
    
    axes[len(folds) - 1].legend(bbox_to_anchor=(1.05, 1), loc=2)
    
    return fig

In [ ]:
fig = visualize_folds(folds, ['alcohol', 'volatile acidity'])
plt.show(fig)

### c) ADALINE evaluation via k-fold cross-validation

In [ ]:
def cross_validate_adaline(folds, features, lr=0.05, epochs=500, mode='batch', verbose=False):
    sum_accuracy = 0

    for i, fold in enumerate(folds):
        X_train = fold[0][features]
        Y_train = fold[0]['goodness']
        
        ad = Adaline(lr=lr)
        train_stats = ad.train(X_train.values, Y_train, epochs, mode, verbose)
        
        X_test = fold[1][features]
        Y_test = fold[1]['goodness']
        accuracy = ad.evaluate_accuracy(X_test.values, Y_test)
        sum_accuracy += accuracy
    
    print('=================================')
    print('Mean model accuracy: {0:.3f}'.format(sum_accuracy / len(folds)))
    print('=================================')

In [ ]:
# Small learning rate and few epochs --> lower accuracy

cross_validate_adaline(folds, ['alcohol', 'volatile acidity'], lr=0.0005,
                       epochs=100, mode='batch', verbose=False)

In [ ]:
# Good learning rate allows to train quickly and gives good accuracy

cross_validate_adaline(folds, ['alcohol', 'volatile acidity'], lr=0.05,
                       epochs=50, mode='batch', verbose=False)

In [ ]:
# And training longer doesn't help

cross_validate_adaline(folds, ['alcohol', 'volatile acidity'], lr=0.05,
                       epochs=500, mode='batch', verbose=False)

In [ ]:
# If learning rate is too big, we will overshoot the point of minimum
# and get lower accuracy

cross_validate_adaline(folds, ['alcohol', 'volatile acidity'], lr=0.5,
                       epochs=200, mode='batch', verbose=False)

---
## V.5 adventures in the Nth dimension

### a) Training on more chemical factors

In [ ]:
features = ['pH', 'alcohol', 'sulphates', 'fixed acidity', 'volatile acidity', 'goodness', 'quality']
selected_wine_data = wine_data[(wine_data['quality'] > 6) | (wine_data['quality'] < 5)][features]
selected_wine_data = selected_wine_data.reset_index(drop=True)

norm_features = ['pH', 'alcohol', 'sulphates', 'fixed acidity', 'volatile acidity']
for feature in norm_features:
    selected_wine_data[feature] = normalize_series(selected_wine_data[feature])

print(selected_wine_data[:10])

In [ ]:
folds = k_fold_split(selected_wine_data, 9)

In [ ]:
# adding 'pH' to taining features

cross_validate_adaline(folds, ['alcohol', 'volatile acidity', 'pH'], lr=0.05,
                       epochs=200, mode='batch', verbose=False)

In [ ]:
# 'pH' -> 'sulphates'

cross_validate_adaline(folds, ['alcohol', 'volatile acidity', 'sulphates'], lr=0.05,
                       epochs=200, mode='batch', verbose=False)

In [ ]:
# + 'fixed acidity'

cross_validate_adaline(folds, ['alcohol', 'volatile acidity', 'fixed acidity', 'sulphates'], lr=0.05,
                       epochs=200, mode='batch', verbose=False)

In [ ]:
# 'fixed acidity' -> 'pH'

cross_validate_adaline(folds, ['alcohol', 'volatile acidity', 'pH', 'sulphates'], lr=0.05,
                       epochs=200, mode='batch', verbose=False)

### b) What does the decision boundary for N factors look like?
Decision boundary for N factors is (N - 1) dimentional hyperplane 

---
# V.6 Marvin's rebuttal

Find a way to successfully classify the Pan-Galactic Gargle Blaster dataset.
Show that your perceptron or ADALINE successfully classifies the Pan-Galactic Gargle Blaster data set by plotting the decision boundary and also show ‘good’ and ‘bad’ Gargle Blaster data points.

In [ ]:
dataset_path = './resources/Pan Galactic Gargle Blaster.csv'

try:
    gargle_blaster_data = pd.read_csv(dataset_path, sep=';')
except FileNotFoundError:
    print('[-] Set `dataset_path` with correct value')

In [ ]:
# Take the gargle blaster as good, if it's quality > 5
gargle_blaster_data = gargle_blaster_data.assign(goodness=pd.Series(gargle_blaster_data['quality'] > 5))

# normalize feature series
gargle_blaster_data['wonderflonium'] = normalize_series(gargle_blaster_data['wonderflonium'])
gargle_blaster_data['fallian marsh gas'] = normalize_series(gargle_blaster_data['fallian marsh gas'])

In [ ]:
fig = plot_scatter_matrix(gargle_blaster_data, 6, 5, True)
plt.show(fig)

Data is not linearly separable, so neither Perceptron, nor ADALINE will not learn from it.

The trick is to transform data points to polar coordinate system, where they will be perfectly separable with a line.

In [ ]:
x1 = gargle_blaster_data.loc[:, 'wonderflonium']
x2 = gargle_blaster_data.loc[:, 'fallian marsh gas']

# compute r and phi, needed for polar coordinates and add them to DataFrame
gargle_blaster_data = gargle_blaster_data.assign(r=pd.Series(x1 * x1 + x2 * x2).pow(1./2))
gargle_blaster_data = gargle_blaster_data.assign(
    phi=pd.Series([math.atan2(x, y) for (x, y) in zip(x1, x2)]))

print(gargle_blaster_data.iloc[:10])
print('...')

In [ ]:
train_set, test_set = train_test_split(gargle_blaster_data, test_size=0.7)

X_train = train_set[['r', 'phi']]
Y_train = train_set['goodness'].values

X_test = test_set[['r', 'phi']]
Y_test = test_set['goodness'].values

In [ ]:
ad = Adaline(lr=0.005)

train_stats = ad.train(X_train.values, Y_train, 70, 'batch', verbose=True)

In [ ]:
ad.evaluate_accuracy(X_test.values, Y_test)

In [ ]:
fig = plot_performance(train_stats, train_set, ['phi', 'r'], 6, 5, -1, False)
plt.show(fig)